In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
from os.path import join
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

import uproot
import hist
import mplhep as hep
from coffea import util
import numpy as np
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty

sys.path.append("../")
#from azh_analysis.utils.plotting import (
#    get_category_labels, get_color_list, 
#)
from azh_analysis.utils.parameters import (
    get_lumis, get_categories, 
)

year, btag = "all", "ggA"

In [ ]:
def unpack_limits(year, btag, category=None):
    masses = np.array([225, 250, 275, 300, 325, 350, 375, 400, 450, 500, 600, 700, 750, 800, 900, 1000, 1200, 1400, 1600, 1800, 2000])
    combine_dir = "/uscms/home/jdezoort/nobackup/combine/CMSSW_11_3_4/src"
    limits = defaultdict(list)

    for mass in masses:
        if category is not None:
            name = f"higgsCombine.{year}_{btag}_{category}.AsymptoticLimits.mH"+str(mass)+".root"
        else: 
            name = f"higgsCombine.{year}_{btag}.AsymptoticLimits.mH"+str(mass)+".root"
        try:
            rootfile = uproot.open(
                join(combine_dir, name)
            )
            limit = rootfile["limit"].arrays()["limit"]
        except: 
            print("skipping", mass)
            continue

        if len(limit)<5: continue
        limits["mass"].append(mass)
        limits["down_2s"].append(limit[0])
        limits["down_1s"].append(limit[1])
        limits["expected"].append(limit[2])
        limits["up_1s"].append(limit[3])
        limits["up_2s"].append(limit[4])
    return limits


In [ ]:
hep.style.use(["CMS", "fira", "firamath"])
colors = {
    "DY": "#0A9396",
    "SM-H(125)": "#E9D8A6",
    "ZZ": "#94D2BD",
    "WZ": "#9b2226",
    "tt": "#EE9B00",
    "VVV": "#bb3e03",
}
limits = unpack_limits(year, btag)


fig, ax = plt.subplots(dpi=800, figsize=(8,8))
ax.fill_between(limits["mass"], limits["down_2s"], limits["up_2s"], color='#ffff01', label="95% Expected")
ax.fill_between(limits["mass"], limits["down_1s"], limits["up_1s"], color='#00ff01', label="68% Expected")
ax.plot(limits["mass"], limits["expected"], linestyle="--", color="black", label="Median Expected")
#ax.plot(limit_obs, "k-", lw=2.5, label="Observed")
ax.set_xlabel(r"$m_A$ [GeV]")
ax.set_ylabel(r"$\sigma$"+f"({btag})"+r"$\times BR(A\rightarrow Zh\rightarrow ll\tau\tau)$ [fb]")
ax.set_xlim([225, 2000])
ax.set_ylim([0,15])
hep.cms.label("Preliminary", data=True, lumi=get_lumis()["Run 2"], year="", ax=ax, fontsize=20)
plt.legend(loc="best", fontsize=15)
plt.tight_layout()
plt.savefig(f"../plots/limits/limits_{year}_{btag}.pdf", dpi=800)
plt.show()

In [ ]:
hep.style.use(["CMS", "fira", "firamath"])

fig, ax = plt.subplots(dpi=120, figsize=(8,8))

limits = unpack_limits(year, btag)
ax.plot(limits["mass"], limits["expected"], marker="", color="black", linestyle="-", label="Combined")
cat_labels = get_category_labels()
for cat in ["eeem", "eeet", "eemt", "eett", "mmem", "mmet", "mmmt", "mmtt"]:
    print(cat)
    limits = unpack_limits(year, btag, category=cat)
    ax.plot(limits["mass"], limits["expected"], marker="", linestyle="--", label=f"{cat_labels[cat]}")


ax.set_xlabel(r"$m_A$ [GeV]")
ax.set_ylabel(r"$\sigma$"+f"({btag})"+r"$\times\mathrm{BR}(A\rightarrow Zh\rightarrow ll\tau\tau)$ [fb]")
ax.set_xlim([225, 2000])
ax.set_ylim([0,80])
hep.cms.label("Preliminary", data=True, lumi=get_lumis()["Run 2"], year="", ax=ax, fontsize=20)
plt.legend(loc="best", fontsize=15)
plt.tight_layout()
plt.savefig(f"../plots/limits/limits_cat_{year}_{btag}.pdf", dpi=800)
plt.show()

In [ ]:
higgsCombine.all_ggA_mmet.AsymptoticLimits.mH1200.root
